# Watershed Algorithm (Part 1)

[Watershed (image processing)](https://en.wikipedia.org/wiki/Watershed_(image_processing))
[Watershed OpenCV - PyImageSearch](https://www.pyimagesearch.com/2015/11/02/watershed-opencv/)
[Image Segmentation with Watershed Algorithm](https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_imgproc/py_watershed/py_watershed.html)

The watershed algorithm transformation treats the image it operates upon like a topographic map, with the brightness of each point representing its height, and finds the lines that run along the tops of ridges. 

Any grayscale image can be viewed as a topographic surface where high intensity denotes peaks and hills while low intensity denotes valleys. The algorithm can then fill every isolated valley (local minima) with different coloured water (labels). We're colouring the different segments of the image.

As the "water" rises, depending on the peaks (gradients) nearby, "water" from different valleys (different segments of the image), with different colours could start to merge. To avoid this merging, the algorithm creates barriers (segment edge boundaries) in locations where "water" merges. A boundary will be created between segments of different brightness levels.

This algorithm is especially useful for segmenting images into background and foreground in situations that are difficult for other algorithms. A common example is the use of coins next to each other on a table. In this case, it may be unclear to the algorithm if it should be treated as one large object or many small objects. The watershed algorithm segments out each coin individually as humans do. 

![Coins on the table](../img/water_result.jpg)


[The Power Watershed Algorithm](http://powerwatershed.sourceforge.net/)

It is possible to provide our own custom "seeds" that allow us to manually start where the valleys of the watersheds go.

![Customs seeds](../img/multi_seeds2.png)

Then the image segments can be calculated:

![Segmentation with Power Watershed algorithm](../img/PW_overlay.gif)

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def display(img, cmap='gray'):
    fig = plt.figure(figsize=(12, 10))
    ax = fig.add_subplot(111)
    ax.imshow(img, cmap='gray')

In [ ]:
sep_coins = cv2.imread('../data/pennies.jpg')

In [ ]:
display(sep_coins)

Lots of computer vision algorithms would treat this as a giant blob (a single object) on a white background. We want to extract 7 segments here actually: 6 for coins and 1 for the background.

Let's first try to apply some of the CV algorithms shown so far and perform a segmentation:
* Median blur - used to remove small, unnecessary details like images on coins
* Convert to grayscale
* Binary threshold - to conver it to black and white
* Find contours

In [ ]:
# choosing larger kernel size as image itself is quite large (3000x4000px)
sep_coins_blur = cv2.medianBlur(sep_coins, 25)

In [ ]:
display(sep_coins_blur)

In [ ]:
sep_coins_gray = cv2.cvtColor(sep_coins_blur, cv2.COLOR_BGR2GRAY)

In [ ]:
display(sep_coins_gray)

In [ ]:
ret, sep_coins_thresh = cv2.threshold(sep_coins_gray, 160, 255, cv2.THRESH_BINARY_INV)

In [ ]:
display(sep_coins_thresh)

In [ ]:
contours, hierarchy = cv2.findContours(sep_coins_thresh.copy(), cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)

In [ ]:
for i in range(len(contours)):
    if hierarchy[0][i][3] == -1:  # is it external contour?
        cv2.drawContours(sep_coins, contours, i, (255, 0, 0), 10) # red contours, thickness = 10
        

In [ ]:
display(sep_coins)

As we can see, contours don't go around each coin but around their union object (external contour of the general, joint shape) which is not what we want.